In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Specific libraries
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge

In [ ]:
# Data loader
variables = ["date", "T_station", "LOC"]

answer = input("Run for Day or night? (d/n): ")

if answer == "d":
# building model for daytime
    print("Selected: daytime")

    answer = input("Run with UHII as resp. variable? (y/n): ")

    # load dataset
    if answer == "y":
        print("Selected: UHII as Y")

        # read data as a Pandas dataframe
        df = pd.read_csv('summer_X_all_daytime_hour_LCZ_UHII.csv')

        # drop specific variables/columns
        df = df.drop(columns=variables)
        
        # drop rows with at least one NaN
        df.dropna(inplace=True)

        # read fixed temporal variables
        temporal = pd.read_csv('summer_X_temporal_daytime_hour_LCZ.csv')

        # drop rows with at least one NaN
        temporal.dropna(inplace=True)

        pred_output = 'summer_RF_UHII_daytime_mean.tif'

    elif answer == "n":
        print("Selected: Air temperature as Y")

        # read data as a Pandas dataframe
        df = pd.read_csv('summer_X_all_daytime_hour_LCZ.csv')

        # drop specific variables/columns
        df = df.drop(columns=variables)
        
        # drop rows with at least one NaN
        df.dropna(inplace=True)

        # check remaining variables
        # print(df.head())

        # read fixed temporal variables
        temporal = pd.read_csv('summer_X_temporal_daytime_hour_LCZ.csv')

        # drop rows with at least one NaN
        temporal.dropna(inplace=True)

        pred_output = 'summer_RF_AirTemp_daytime_mean.tif'

    else:
        print("Invalid option")

elif answer == "n":
# building model for nighttime
    print("Selected: nighttime")

    answer = input("Run with UHII as resp. variable? (y/n): ")

    # load dataset
    if answer == "y":
        print("Selected: UHII as Y")
        
        # read data as a Pandas dataframe
        df = pd.read_csv('summer_X_all_nighttime_hour_LCZ_UHII.csv')
        
        # drop specific variables/columns
        df = df.drop(columns=variables)

        # drop rows with at least one NaN
        df.dropna(inplace=True)

        # read fixed temporal variables
        temporal = pd.read_csv('summer_X_temporal_nighttime_hour_LCZ.csv')

        # drop rows with at least one NaN
        temporal.dropna(inplace=True)

        pred_output = 'summer_RF_UHII_nighttime_mean.tif'

    elif answer == "n":
        print("Selected: Air temperature as Y")

        # read data as a Pandas dataframe
        df = pd.read_csv('summer_X_all_nighttime_hour_LCZ.csv')

        # drop specific variables/columns
        df = df.drop(columns=variables)
        
        # drop rows with at least one NaN
        df.dropna(inplace=True)

        # read fixed temporal variables
        temporal = pd.read_csv('summer_X_temporal_nighttime_hour_LCZ.csv')

        # drop rows with at least one NaN
        temporal.dropna(inplace=True)

        pred_output = 'summer_RF_AirTemp_nighttime_mean.tif'

    else:
        print("Invalid option")

else:
    print("Invalid option")

In [ ]:
# Read the data cube which contains all stacked rasters/variables
with rasterio.open('ICALON_Rasters_MergedCube.tif') as src:
    # Read all bands of the raster into a numpy array
    spatial_variables = src.read(list(range(1, src.count+1)))

In [ ]:
# Run using the temporal variables corresponding to each cluster day/hour

# Calculate using a specific day and hour
temporal['date'] = pd.to_datetime(temporal['date'], format='%d/%m/%Y %H:%M')  # convert to datetime format

dates_to_match = [
    '17/07/2011 06:00', '17/07/2011 12:00', '17/07/2011 18:00', '17/07/2011 23:00', # C1
    '06/07/2011 06:00', '06/07/2011 12:00', '06/07/2011 18:00', '06/07/2011 23:00', # C2
    '01/07/2011 06:00', '01/07/2011 12:00', '01/07/2011 18:00', '01/07/2011 23:00', # C3
                ]

masks = [temporal['date'] == pd.to_datetime(date, format='%d/%m/%Y %H:%M') for date in dates_to_match]

temporal_variables = pd.concat([temporal[mask] for mask in masks], axis=0).iloc[:, 1:].values.T

# Reshape so it fits the 3-D cube
temporal_variables_reshape = np.reshape(temporal_variables[:, 0], (temporal_variables.shape[0], 1, 1))
temporal_variables_tiled = np.tile(temporal_variables_reshape, (1, spatial_variables.shape[1], spatial_variables.shape[2]))

# Concatenate the spatial data cube with the temporal (fixed) variables
big_X = np.concatenate((spatial_variables, temporal_variables_tiled), axis=0)

# Cleanup the RAM
%reset_selective -f temporal_variables_reshape temporal_variables_tiled spatial_variables

In [ ]:
# Load the trained model
from joblib import load

mdl = load('summer_RF_UHII_daytime.joblib')
mdl = load('summer_RF_UHII_nighttime.joblib')

In [ ]:
# Reshape X to a flat vector before predicting the air temperature (or UHII)
from sklearn.impute import SimpleImputer

reshaped_X = np.zeros((big_X.shape[1]*big_X.shape[2], big_X.shape[0]))

k = 0
for j in range(big_X.shape[2]):
    for i in range(big_X.shape[1]):
        reshaped_X[k, :] = big_X[:, i, j]
        k += 1

imputer = SimpleImputer(strategy='mean')
reshaped_X = imputer.fit_transform(reshaped_X)

# Predict the air temperature (or UHII) across the entire AOI
flat_preds = mdl.predict(reshaped_X)

# Reshape "flat_preds" to the same shape of "big_X"
Y_preds = np.zeros((big_X.shape[1], big_X.shape[2]))
k = 0
for j in range(big_X.shape[2]):
    for i in range(big_X.shape[1]):
        Y_preds[i, j] = flat_preds[k]
        k += 1

In [ ]:
# Save simulations as GeoTIFF
import os
import rasterio

# Open the reference GeoTIFF file (to copy the SRC)
with rasterio.open('aoi.tif') as src:
    profile = src.profile

# Update the metadata to match the shape of the output array
profile.update(
        count=1,
        height=Y_preds.shape[0],
        width=Y_preds.shape[1],
        dtype=np.float32,
        compress='lzw',
        predictor=2,
        nodata=np.nan)

# Write the output array to a new GeoTIFF file
with rasterio.open(pred_output, 'w', **profile) as dst:
    dst.write(Y_preds, 1)